# OPERA CSLC-S1 Data Access Notebook

## 1. Import needed Python packages

In [ ]:
from getpass import getpass
from pathlib import Path
from pprint import pprint
import re

import asf_search as disco

## 2. Authenticate with asf_search and start a session

**Gather credentials for authentication with Earth Data Login**

In [ ]:
username = input("Enter your EDL username")
password = getpass("Enter your EDL password")

**Start an asf_search session**

In [ ]:
try:
    user_pass_session = disco.ASFSession().auth_with_creds(username, password)
except disco.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

## 3. Create a directory in which to download the CSLCs

**Create a data directory alongside the Jupyter Book containing this notebook** 

In [ ]:
name = input("Enter the name of a directory to create, which will hold your downloaded CSLCs")
data_dir = Path.cwd().parents[1]/name
data_dir.mkdir(exist_ok=True)
print(f"Data Directory: {data_dir}")

## 4. Search for OPERA CSLC-S1 products

**Option 1: List Search**

- Search by OPERA product ID

- Identify OPERA IDs with an [OPERA-S1 Geographic Search on Vertex](https://search.asf.alaska.edu/#/?maxResults=250&dataset=OPERA-S1)


In [ ]:
product_list = [
    "OPERA_L2_CSLC-S1_T173-370304-IW2_20231018T134413Z_20231019T180049Z_S1A_VV_v1.0",
    "OPERA_L2_CSLC-S1_T173-370304-IW2_20231205T134412Z_20231206T075651Z_S1A_VV_v1.0"
    ]
results = disco.granule_search(product_list)
# print(results)

**Option 2: Geographic Search**
- Search by intersection with an area of interest
  - Define in WKT as a POINT or POLYGON
    - `'intersectsWith': 'POINT(-117.4231 35.7597)'`
    - `'intersectsWith': 'POLYGON((-114.9066 35.2997,-114.159 35.2997,-114.159 35.8598,-114.9066 35.8598,-114.9066 35.2997))'`

In [ ]:
options = {
    # 'intersectsWith': 'POINT(-121.643 38.127)',
    'dataset': 'OPERA-S1',
    # 'start': '2019-07-03T00:00:00Z',
    # 'end': '2023-12-19T00:00:00Z',
    # 'flightDirection': 'ASCENDING',
    'operaBurstID': 'T035_073252_IW3',
    'processingLevel': [
        # 'CSLC',
        'CSLC-STATIC'
    ],
    'maxResults': '1000'
}

results = disco.search(**options)
pprint([r.properties['fileName'] for r in results])

## 5. Filter the Results

- The geographic search may return current and superceded data
- When >1 CSLC are found for a burst/acquisition date, select the most recently processed

In [ ]:
filtered_bursts = dict()
acquisition_date_regex = r"(?<=OPERA_L2_CSLC-S1_)T\d{3}-\d{6}-IW\d_\d{8}T\d{6}Z(?=_\d{8}T\d{6}Z)"
process_date_regex = '(?<=OPERA_L2_CSLC-S1_T\d{3}-\d{6}-IW\d_\d{8}T\d{6}Z_)\d{8}T\d{6}Z'

for b in results:
    filename = b.properties['fileName']
    try:
        id_date = re.search(acquisition_date_regex, filename).group(0)
        try:
            # for scenes that only differ by processing date, we can use a simple relational comparisons
            if filtered_bursts[id_date].properties['fileName'] < filename:
                filtered_bursts[id_date] = b
        except KeyError:
            filtered_bursts[id_date] = b
    except AttributeError:
        raise Exception(f"Acquisition not found in filename: {str(b)}")
        
filtered_bursts = list(filtered_bursts.values())   

for b in filtered_bursts:
    print(b.properties['fileName'])

## 6. Download Data

In [ ]:
for burst in filtered_bursts:
    burst.download(data_dir, session=user_pass_session)

In [ ]:
print(results[0])

In [ ]:
for burst in results:
    burst.download(data_dir, session=user_pass_session)